<a href="https://colab.research.google.com/github/Chaitanya-Aggarwal/widsSubmission/blob/master/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/
%ls

/content/drive/MyDrive
'Colab Notebooks'/   models/        test_model.py     vgg.py
 ebb_dataset/        __pycache__/   train_model.py    visual_samples/
 load_dataset.py     random.py      Untitled0.ipynb
 main.py             random.txt     utils.py
 master.ipynb        results/       vgg_pretrained/


In [7]:
# Copyright 2020 by Andrey Ignatov. All Rights Reserved.

# *****************************LOAD DATASET*************************************************************

from __future__ import print_function
from scipy import misc
from PIL import Image
import imageio
import os
import numpy as np
import tensorflow as tf
import sys
import scipy.io
import imageio
#*************************************VGG****************************************************
IMAGE_MEAN = np.array([123.68,  116.779,  103.939])


def _conv_layer_(input, weights, bias):
    print("no errors here")
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1), padding='SAME')
    return tf.nn.bias_add(conv, bias)

def net(path_to_vgg_net, input_image):

    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    data = scipy.io.loadmat(path_to_vgg_net)
    weights = data['layers'][0]

    net = {}
    current = input_image
    for i, name in enumerate(layers):
        layer_type = name[:4]
        if layer_type == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer_(current, kernels, bias)
        elif layer_type == 'relu':
            current = tf.nn.relu(current)
        elif layer_type == 'pool':
            current = _pool_layer(current)
        net[name] = current

    return net



def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME')


def preprocess(image):
    return image - IMAGE_MEAN

#*************************************UTILS*************************************************

# Copyright 2020 by Andrey Ignatov. All Rights Reserved.

from functools import reduce
import tensorflow as tf
import numpy as np
import sys
import os

NUM_DEFAULT_TRAIN_ITERS = [-1, 100000, 80000, 30000, 20000, 20000, 5000, 5000]


def process_command_args(arguments):

    # Specifying the default parameters

    level = 1
    batch_size = 50

    train_size = 4894
    learning_rate = 5e-5

    eval_step = 1000
    restore_iter = None
    num_train_iters = None

    dataset_dir = 'ebb_dataset/'
    vgg_dir = 'vgg_pretrained/imagenet-vgg-verydeep-19.mat'

    for args in arguments:

        if args.startswith("level"):
            level = int(args.split("=")[1])

        if args.startswith("batch_size"):
            batch_size = int(args.split("=")[1])

        if args.startswith("train_size"):
            train_size = int(args.split("=")[1])

        if args.startswith("learning_rate"):
            learning_rate = float(args.split("=")[1])

        if args.startswith("restore_iter"):
            restore_iter = int(args.split("=")[1])

        if args.startswith("num_train_iters"):
            num_train_iters = int(args.split("=")[1])

        # -----------------------------------

        if args.startswith("dataset_dir"):
            dataset_dir = args.split("=")[1]

        if args.startswith("vgg_dir"):
            vgg_dir = args.split("=")[1]

        if args.startswith("eval_step"):
            eval_step = int(args.split("=")[1])

    if restore_iter is None and level < 7:
        restore_iter = get_last_iter(level + 1)
        if restore_iter == -1:
            print("Error: Cannot find any pre-trained models for PyNET's level " + str(level + 1) + ".")
            print("Aborting the training.")
            sys.exit()

    if num_train_iters is None:
        num_train_iters = NUM_DEFAULT_TRAIN_ITERS[level]

    print("The following parameters will be applied for CNN training:")

    print("Training level: " + str(level))
    print("Batch size: " + str(batch_size))
    print("Learning rate: " + str(learning_rate))
    print("Training iterations: " + str(num_train_iters))
    print("Evaluation step: " + str(eval_step))
    print("Restore Iteration: " + str(restore_iter))
    print("Path to the dataset: " + dataset_dir)
    print("Path to VGG-19 network: " + vgg_dir)

    return level, batch_size, train_size, learning_rate, restore_iter, num_train_iters,\
           dataset_dir, vgg_dir, eval_step


def process_test_model_args(arguments):

    level = 1
    restore_iter = None

    dataset_dir = 'ebb_dataset/'
    use_gpu = "true"

    orig_model = "false"

    for args in arguments:

        if args.startswith("level"):
            level = int(args.split("=")[1])

        if args.startswith("dataset_dir"):
            dataset_dir = args.split("=")[1]

        if args.startswith("restore_iter"):
            restore_iter = int(args.split("=")[1])

        if args.startswith("use_gpu"):
            use_gpu = args.split("=")[1]

        if args.startswith("orig"):
            orig_model = args.split("=")[1]

    if restore_iter is None and orig_model == "false":
        restore_iter = get_last_iter(level)
        if restore_iter == -1:
            print("Error: Cannot find any pre-trained models for PyNET's level " + str(level) + ".")
            sys.exit()

    return level, restore_iter, dataset_dir, use_gpu, orig_model


def get_last_iter(level):

    saved_models = [int((model_file.split("_")[-1]).split(".")[0])
                    for model_file in os.listdir("models/")
                    if model_file.startswith("pynet_level_" + str(level))]

    if len(saved_models) > 0:
        return np.max(saved_models)
    else:
        return -1


def log10(x):
    numerator = tf.compat.v1.log(x)
    denominator = tf.compat.v1.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator


def _tensor_size(tensor):
    from operator import mul
    return reduce(mul, (d.value for d in tensor.get_shape()[1:]), 1)

#*************************************UTILS*************************************************



#********************************LOAD DATASET************************************************************
def load_test_data(dataset_dir, PATCH_WIDTH, PATCH_HEIGHT, DSLR_SCALE):

    test_directory_orig = dataset_dir + 'test/original/'
    test_directory_orig_depth = dataset_dir + 'test/original_depth/'
    test_directory_blur = dataset_dir + 'test/bokeh/'

    NUM_TEST_IMAGES = 48
    # NUM_TEST_IMAGES = len([name for name in os.listdir(test_directory_orig)
                          #  if os.path.isfile(os.path.join(test_directory_orig, name))])

    test_data = np.zeros((NUM_TEST_IMAGES, PATCH_HEIGHT, PATCH_WIDTH, 4))
    test_answ = np.zeros((NUM_TEST_IMAGES, int(PATCH_HEIGHT * DSLR_SCALE), int(PATCH_WIDTH * DSLR_SCALE), 3))

    for i in range(0, NUM_TEST_IMAGES):
        i += 2001
        I = imageio.imread(test_directory_orig + str(i) + '.jpg')
        I_depth = imageio.imread(test_directory_orig_depth + str(i) + '.jpg')

        # Downscaling the image by a factor of 2
        I = imageio.imresize(I, 0.5, interp='bicubic')

        # Making sure that its width is multiple of 32
        new_width = int(I.shape[1]/32) * 32
        I = I[:, 0:new_width, :]

        # Stacking the image together with its depth map
        I_temp = np.zeros((I.shape[0], I.shape[1], 4))
        I_temp[:, :, 0:3] = I
        I_temp[:, :, 3] = I_depth
        I = I_temp

        h, w, d = I.shape
        y = np.random.randint(0, w - 512)

        # Extracting random patch of width PATCH_WIDTH
        I = np.float32(I[:, y:y + PATCH_WIDTH, :]) / 255.0
        test_data[i, :] = I

        I = imageio.imread(test_directory_blur + str(i) + '.jpg')
        I = np.float32(imageio.imresize(I[:, y*2:y*2 + 1024, :], DSLR_SCALE / 2, interp='bicubic')) / 255.0
        test_answ[i, :] = I
        i-=2001
    return test_data, test_answ


def load_training_batch(dataset_dir, PATCH_WIDTH, PATCH_HEIGHT, DSLR_SCALE, train_size):

    test_directory_orig = dataset_dir + 'train/original/'
    test_directory_orig_depth = dataset_dir + 'train/original_depth/'
    test_directory_blur = dataset_dir + 'train/bokeh/'

    NUM_TRAINING_IMAGES = 500
    # NUM_TRAINING_IMAGES = len([name for name in os.listdir(test_directory_orig)
                          #  if os.path.isfile(os.path.join(test_directory_orig, name))])

    TRAIN_IMAGES = np.random.choice(np.arange(0, NUM_TRAINING_IMAGES), train_size, replace=False)

    test_data = np.zeros((train_size, PATCH_HEIGHT, PATCH_WIDTH, 4))
    test_answ = np.zeros((train_size, int(PATCH_HEIGHT * DSLR_SCALE), int(PATCH_WIDTH * DSLR_SCALE), 3))

    i = 0
    for img in TRAIN_IMAGES:

        I = imageio.imread(test_directory_orig + str(img) + '.jpg')
        I_depth = imageio.imread(test_directory_orig_depth + str(img) + '.png')

        # Downscaling the image by a factor of 2
        I = imageio.imresize(I, 0.5, interp='bicubic')

        # Making sure that its width is multiple of 32
        new_width = int(I.shape[1] / 32) * 32
        I = I[:, 0:new_width, :]

        # Stacking the image together with its depth map
        I_temp = np.zeros((I.shape[0], I.shape[1], 4))
        I_temp[:, :, 0:3] = I
        I_temp[:, :, 3] = I_depth
        I = I_temp

        h, w, d = I.shape
        y = np.random.randint(0, w - 512)

        # Extracting random patch of width PATCH_WIDTH
        I = np.float32(I[:, y:y + PATCH_WIDTH, :]) / 255.0
        test_data[i, :] = I

        I = imageio.imread(test_directory_blur + str(img) + '.jpg')
        I = np.float32(imageio.imresize(I[:, y * 2:y * 2 + 1024, :], DSLR_SCALE / 2, interp='bicubic')) / 255.0
        test_answ[i, :] = I

        i += 1

    return test_data, test_answ


def load_input_image(image_dir, depth_maps_dir, photo):

    I = imageio.imread(image_dir + photo)
    I_depth = imageio.imread(depth_maps_dir + str(photo.split(".")[0]) + '.png')

    # Downscaling the image by a factor of 2
    I = imageio.imresize(I, 0.5, interp='bicubic')

    # Making sure that its width is multiple of 32
    new_width = int(I.shape[1] / 32) * 32
    I = I[:, 0:new_width, :]
    I_depth = I_depth[:, 0:new_width]

    # Stacking the image together with its depth map
    I_temp = np.zeros((I.shape[0], I.shape[1], 4))
    I_temp[:, :, 0:3] = I
    I_temp[:, :, 3] = I_depth

    I = np.float32(I_temp) / 255.0
    I = np.reshape(I, [1, I.shape[0], I.shape[1], 4])

    return I
#********************************LOAD DATASET************************************************************


#********************************MODEL.py************************************************************
def PyNET(input, instance_norm=True, instance_norm_level_1=False):

    # Note: the paper uses a different layer naming scheme.
    # In this code, layer N corresponds to layer N+2 from the article.

    with tf.compat.v1.variable_scope("generator"):

        # -----------------------------------------
        # Space-to-depth layer

        space2depth_l0 = tf.nn.space_to_depth(input, 2)                                         # 512 -> 256

        # -----------------------------------------
        # Downsampling layers

        conv_l1_d1 = _conv_multi_block(space2depth_l0, 3, num_maps=32, instance_norm=False)     # 256 -> 256
        pool1 = max_pool(conv_l1_d1, 2)                                                         # 256 -> 128

        conv_l2_d1 = _conv_multi_block(pool1, 3, num_maps=64, instance_norm=instance_norm)      # 128 -> 128
        pool2 = max_pool(conv_l2_d1, 2)                                                         # 128 -> 64

        conv_l3_d1 = _conv_multi_block(pool2, 3, num_maps=128, instance_norm=instance_norm)     # 64 -> 64
        pool3 = max_pool(conv_l3_d1, 2)                                                         # 64 -> 32

        conv_l4_d1 = _conv_multi_block(pool3, 3, num_maps=256, instance_norm=instance_norm)     # 32 -> 32
        pool4 = max_pool(conv_l4_d1, 2)                                                         # 32 -> 16

        # -----------------------------------------
        # Processing: Level 5,  Input size: 16 x 16

        conv_l5_d1 = _conv_multi_block(pool4, 3, num_maps=512, instance_norm=instance_norm)
        conv_l5_d2 = _conv_multi_block(conv_l5_d1, 3, num_maps=512, instance_norm=instance_norm) + conv_l5_d1
        conv_l5_d3 = _conv_multi_block(conv_l5_d2, 3, num_maps=512, instance_norm=instance_norm) + conv_l5_d2
        conv_l5_d4 = _conv_multi_block(conv_l5_d3, 3, num_maps=512, instance_norm=instance_norm)

        conv_t4a = _conv_tranpose_layer(conv_l5_d4, 256, 3, 2)      # 16 -> 32
        conv_t4b = _conv_tranpose_layer(conv_l5_d4, 256, 3, 2)      # 16 -> 32

        # -> Output: Level 5

        conv_l5_out = _conv_layer(conv_l5_d4, 3, 3, 1, relu=False, instance_norm=False)
        output_l5 = tf.nn.tanh(conv_l5_out) * 0.58 + 0.5

        # -----------------------------------------
        # Processing: Level 4,  Input size: 32 x 32

        conv_l4_d2 = stack(conv_l4_d1, conv_t4a)
        conv_l4_d3 = _conv_multi_block(conv_l4_d2, 3, num_maps=256, instance_norm=instance_norm)
        conv_l4_d4 = _conv_multi_block(conv_l4_d3, 3, num_maps=256, instance_norm=instance_norm) + conv_l4_d3
        conv_l4_d5 = _conv_multi_block(conv_l4_d4, 3, num_maps=256, instance_norm=instance_norm) + conv_l4_d4
        conv_l4_d6 = stack(_conv_multi_block(conv_l4_d5, 3, num_maps=256, instance_norm=instance_norm), conv_t4b)

        conv_l4_d7 = _conv_multi_block(conv_l4_d6, 3, num_maps=256, instance_norm=instance_norm)

        conv_t3a = _conv_tranpose_layer(conv_l4_d7, 128, 3, 2)      # 32 -> 64
        conv_t3b = _conv_tranpose_layer(conv_l4_d7, 128, 3, 2)      # 32 -> 64

        # -> Output: Level 4

        conv_l4_out = _conv_layer(conv_l4_d7, 3, 3, 1, relu=False, instance_norm=False)
        output_l4 = tf.nn.tanh(conv_l4_out) * 0.58 + 0.5

        # -----------------------------------------
        # Processing: Level 3,  Input size: 64 x 64

        conv_l3_d2 = stack(conv_l3_d1, conv_t3a)
        conv_l3_d3 = _conv_multi_block(conv_l3_d2, 5, num_maps=128, instance_norm=instance_norm) + conv_l3_d2
        conv_l3_d4 = _conv_multi_block(conv_l3_d3, 5, num_maps=128, instance_norm=instance_norm) + conv_l3_d3
        conv_l3_d5 = _conv_multi_block(conv_l3_d4, 5, num_maps=128, instance_norm=instance_norm) + conv_l3_d4
        conv_l3_d6 = stack(_conv_multi_block(conv_l3_d5, 5, num_maps=128, instance_norm=instance_norm), conv_l3_d1)
        conv_l3_d7 = stack(conv_l3_d6, conv_t3b)

        conv_l3_d8 = _conv_multi_block(conv_l3_d7, 3, num_maps=128, instance_norm=instance_norm)

        conv_t2a = _conv_tranpose_layer(conv_l3_d8, 64, 3, 2)       # 64 -> 128
        conv_t2b = _conv_tranpose_layer(conv_l3_d8, 64, 3, 2)       # 64 -> 128

        # -> Output: Level 3

        conv_l3_out = _conv_layer(conv_l3_d8, 3, 3, 1, relu=False, instance_norm=False)
        output_l3 = tf.nn.tanh(conv_l3_out) * 0.58 + 0.5

        # -------------------------------------------
        # Processing: Level 2,  Input size: 128 x 128

        conv_l2_d2 = stack(conv_l2_d1, conv_t2a)
        conv_l2_d3 = stack(_conv_multi_block(conv_l2_d2, 5, num_maps=64, instance_norm=instance_norm), conv_l2_d1)

        conv_l2_d4 = _conv_multi_block(conv_l2_d3, 7, num_maps=64, instance_norm=instance_norm) + conv_l2_d3
        conv_l2_d5 = _conv_multi_block(conv_l2_d4, 7, num_maps=64, instance_norm=instance_norm) + conv_l2_d4
        conv_l2_d6 = _conv_multi_block(conv_l2_d5, 7, num_maps=64, instance_norm=instance_norm) + conv_l2_d5
        conv_l2_d7 = stack(_conv_multi_block(conv_l2_d6, 7, num_maps=64, instance_norm=instance_norm), conv_l2_d1)

        conv_l2_d8 = stack(_conv_multi_block(conv_l2_d7, 5, num_maps=64, instance_norm=instance_norm), conv_t2b)
        conv_l2_d9 = _conv_multi_block(conv_l2_d8, 3, num_maps=64, instance_norm=instance_norm)

        conv_t1a = _conv_tranpose_layer(conv_l2_d9, 32, 3, 2)       # 128 -> 256
        conv_t1b = _conv_tranpose_layer(conv_l2_d9, 32, 3, 2)       # 128 -> 256

        # -> Output: Level 2

        conv_l2_out = _conv_layer(conv_l2_d9, 3, 3, 1, relu=False, instance_norm=False)
        output_l2 = tf.nn.tanh(conv_l2_out) * 0.58 + 0.5

        # -------------------------------------------
        # Processing: Level 1,  Input size: 256 x 256

        conv_l1_d2 = stack(conv_l1_d1, conv_t1a)
        conv_l1_d3 = stack(_conv_multi_block(conv_l1_d2, 5, num_maps=32, instance_norm=False), conv_l1_d1)

        conv_l1_d4 = _conv_multi_block(conv_l1_d3, 7, num_maps=32, instance_norm=False)

        conv_l1_d5 = _conv_multi_block(conv_l1_d4, 9, num_maps=32, instance_norm=instance_norm_level_1)
        conv_l1_d6 = _conv_multi_block(conv_l1_d5, 9, num_maps=32, instance_norm=instance_norm_level_1) + conv_l1_d5
        conv_l1_d7 = _conv_multi_block(conv_l1_d6, 9, num_maps=32, instance_norm=instance_norm_level_1) + conv_l1_d6
        conv_l1_d8 = _conv_multi_block(conv_l1_d7, 9, num_maps=32, instance_norm=instance_norm_level_1) + conv_l1_d7

        conv_l1_d9 = stack(_conv_multi_block(conv_l1_d8, 7, num_maps=32, instance_norm=False), conv_l1_d1)

        conv_l1_d10 = stack(_conv_multi_block(conv_l1_d9, 5, num_maps=32, instance_norm=False), conv_t1b)
        conv_l1_d11 = stack(conv_l1_d10, conv_l1_d1)

        conv_l1_d12 = _conv_multi_block(conv_l1_d11, 3, num_maps=32, instance_norm=False)

        # -> Output: Level 1

        conv_l1_out = _conv_layer(conv_l1_d12, 3, 3, 1, relu=False, instance_norm=False)
        output_l1 = tf.nn.tanh(conv_l1_out) * 0.58 + 0.5

        # ----------------------------------------------------------
        # Processing: Level 0 (x2 upscaling),  Input size: 256 x 256

        conv_l0 = _conv_tranpose_layer(conv_l1_d12, 8, 3, 2)        # 256 -> 512
        conv_l0_out = _conv_layer(conv_l0, 3, 3, 1, relu=False, instance_norm=False)
        output_l0 = tf.nn.tanh(conv_l0_out) * 0.58 + 0.5

        # ----------------------------------------------------------
        # Processing: Level Up (x4 upscaling),  Input size: 512 x 512

        conv_l_up = _conv_tranpose_layer(conv_l0_out, 3, 3, 2)  # 512 -> 1024
        conv_l_up_out = _conv_layer(conv_l_up, 3, 3, 1, relu=False, instance_norm=False)

        output_l_up = tf.nn.tanh(conv_l_up_out) * 0.58 + 0.5

    return output_l_up, output_l0, output_l1, output_l2, output_l3, output_l4, output_l5


def _conv_multi_block(input, max_size, num_maps, instance_norm):

    conv_3a = _conv_layer(input, num_maps, 3, 1, relu=True, instance_norm=instance_norm)
    conv_3b = _conv_layer(conv_3a, num_maps, 3, 1, relu=True, instance_norm=instance_norm)

    output_tensor = conv_3b

    if max_size >= 5:

        conv_5a = _conv_layer(input, num_maps, 5, 1, relu=True, instance_norm=instance_norm)
        conv_5b = _conv_layer(conv_5a, num_maps, 5, 1, relu=True, instance_norm=instance_norm)

        output_tensor = stack(output_tensor, conv_5b)

    if max_size >= 7:

        conv_7a = _conv_layer(input, num_maps, 7, 1, relu=True, instance_norm=instance_norm)
        conv_7b = _conv_layer(conv_7a, num_maps, 7, 1, relu=True, instance_norm=instance_norm)

        output_tensor = stack(output_tensor, conv_7b)

    if max_size >= 9:

        conv_9a = _conv_layer(input, num_maps, 9, 1, relu=True, instance_norm=instance_norm)
        conv_9b = _conv_layer(conv_9a, num_maps, 9, 1, relu=True, instance_norm=instance_norm)

        output_tensor = stack(output_tensor, conv_9b)

    return output_tensor


def stack(x, y):
    return tf.concat([x, y], 3)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def leaky_relu(x, alpha=0.2):
    return tf.maximum(alpha * x, x)


def _conv_layer(net, num_filters, filter_size, strides, relu=True, instance_norm=False, padding='SAME'):

    weights_init = _conv_init_vars(net, num_filters, filter_size)
    strides_shape = [1, strides, strides, 1]
    bias = tf.Variable(tf.constant(0.01, shape=[num_filters]))

    net = tf.nn.conv2d(net, weights_init, strides_shape, padding=padding) + bias

    if instance_norm:
        net = _instance_norm(net)

    if relu:
        net = leaky_relu(net)

    return net


def _instance_norm(net):

    batch, rows, cols, channels = [i.value for i in net.get_shape()]
    var_shape = [channels]

    mu, sigma_sq = tf.compat.v1.nn.moments(net, [1,2], keep_dims=True)
    shift = tf.Variable(tf.zeros(var_shape))
    scale = tf.Variable(tf.ones(var_shape))

    epsilon = 1e-3
    normalized = (net-mu)/(sigma_sq + epsilon)**(.5)

    return scale * normalized + shift


def _conv_init_vars(net, out_channels, filter_size, transpose=False):

    _, rows, cols, in_channels = [i.value for i in net.get_shape()]

    if not transpose:
        weights_shape = [filter_size, filter_size, in_channels, out_channels]
    else:
        weights_shape = [filter_size, filter_size, out_channels, in_channels]

    weights_init = tf.Variable(tf.compat.v1.truncated_normal(weights_shape, stddev=0.01, seed=1), dtype=tf.float32)
    return weights_init


def _conv_tranpose_layer(net, num_filters, filter_size, strides):
    weights_init = _conv_init_vars(net, num_filters, filter_size, transpose=True)

    net_shape = tf.shape(net)
    tf_shape = tf.stack([net_shape[0], net_shape[1] * strides, net_shape[2] * strides, num_filters])

    strides_shape = [1, strides, strides, 1]
    net = tf.nn.conv2d_transpose(net, weights_init, tf_shape, strides_shape, padding='SAME')

    return leaky_relu(net)


def max_pool(x, n):
    return tf.nn.max_pool(x, ksize=[1, n, n, 1], strides=[1, n, n, 1], padding='VALID')
#********************************MODEL.py************************************************************



#********************************train_model.py************************************************************

tf.compat.v1.disable_v2_behavior()

# Processing command arguments

# LEVEL, batch_size, train_size, learning_rate, restore_iter, num_train_iters, dataset_dir, vgg_dir, eval_step = \
    # process_command_args(sys.argv)

for LEVEL in range(7,1,-1):
  batch_size = 20
  train_size = 2000
  learning_rate = 5e-5
  restore_iter = None
  num_train_iters = 5000
  dataset_dir = "ebb_dataset/"
  vgg_dir = "vgg_pretrained/imagenet-vgg-verydeep-19.mat"
  eval_step = 1000

  # Defining the size of the input and target image patches

  PATCH_WIDTH, PATCH_HEIGHT = 512, 512
  DSLR_SCALE = float(1) / (2 ** (LEVEL - 2))

  TARGET_WIDTH = int(PATCH_WIDTH * DSLR_SCALE)
  TARGET_HEIGHT = int(PATCH_HEIGHT * DSLR_SCALE)
  TARGET_DEPTH = 3
  TARGET_SIZE = TARGET_WIDTH * TARGET_HEIGHT * TARGET_DEPTH

  np.random.seed(0)

  # Defining the model architecture

  with tf.Graph().as_default(), tf.compat.v1.Session() as sess:
      
      # Placeholders for training data

      input_ = tf.compat.v1.placeholder(tf.float32, [batch_size, PATCH_HEIGHT, PATCH_WIDTH, 4])
      target_ = tf.compat.v1.placeholder(tf.float32, [batch_size, TARGET_HEIGHT, TARGET_WIDTH, TARGET_DEPTH])

      # Get the rendered bokeh image

      output_l1, output_l2, output_l3, output_l4, output_l5, output_l6, output_l7 = \
          PyNET(input_, instance_norm=True, instance_norm_level_1=False)

      if LEVEL == 7:
          bokeh_img = output_l7
      if LEVEL == 6:
          bokeh_img = output_l6
      if LEVEL == 5:
          bokeh_img = output_l5
      if LEVEL == 4:
          bokeh_img = output_l4
      if LEVEL == 3:
          bokeh_img = output_l3
      if LEVEL == 2:
          bokeh_img = output_l2
      if LEVEL == 1:
          bokeh_img = output_l1

      # Losses

      bokeh_img_flat = tf.reshape(bokeh_img, [-1, TARGET_SIZE])
      target_flat = tf.reshape(target_, [-1, TARGET_SIZE])

      # MSE loss
      loss_mse = tf.reduce_sum(tf.pow(target_flat - bokeh_img_flat, 2)) / (TARGET_SIZE * batch_size)

      # PSNR loss
      loss_psnr = 20 * log10(1.0 / tf.sqrt(loss_mse))

      # SSIM loss
      loss_ssim = tf.reduce_mean(tf.image.ssim(bokeh_img, target_, 1.0))

      # MS-SSIM loss
      loss_ms_ssim = tf.reduce_mean(tf.image.ssim_multiscale(bokeh_img, target_, 1.0))

      # L1 loss
      loss_l1 = tf.compat.v1.losses.absolute_difference(bokeh_img, target_)

      # Content loss
      CONTENT_LAYER = 'relu5_4'

      bokeh_img_vgg = net(vgg_dir, preprocess(bokeh_img * 255))
      target_vgg = net(vgg_dir, preprocess(target_ * 255))

      content_size = _tensor_size(target_vgg[CONTENT_LAYER]) * batch_size
      loss_content = 2 * tf.nn.l2_loss(bokeh_img_vgg[CONTENT_LAYER] - target_vgg[CONTENT_LAYER]) / content_size

      # Final loss function

      if LEVEL > 1:
          loss_generator = loss_l1 * 100
      else:
          loss_generator = loss_l1 * 10 + loss_content * 0.1 + (1 - loss_ssim) * 10

      # Optimize network parameters

      generator_vars = [v for v in tf.compat.v1.global_variables() if v.name.startswith("generator")]
      train_step_gen = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(loss_generator)

      # Initialize and restore the variables

      print("Initializing variables")
      sess.run(tf.compat.v1.global_variables_initializer())

      saver = tf.compat.v1.train.Saver(var_list=generator_vars, max_to_keep=100)

      if LEVEL < 7:
          print("Restoring Variables")
          saver.restore(sess, "models/pynet_level_" + str(LEVEL + 1) + "_iteration_" + str(restore_iter) + ".ckpt")

      saver = tf.compat.v1.train.Saver(var_list=generator_vars, max_to_keep=100)

      # Loading training and test data

      print("Loading test data...")
      test_data, test_answ = load_test_data(dataset_dir, PATCH_WIDTH, PATCH_HEIGHT, DSLR_SCALE)
      print("Test data was loaded\n")

      print("Loading training data...")
      train_data, train_answ = load_training_batch(dataset_dir, PATCH_WIDTH, PATCH_HEIGHT, DSLR_SCALE, train_size)
      print("Training data was loaded\n")

      TEST_SIZE = test_data.shape[0]
      num_test_batches = int(test_data.shape[0] / batch_size)

      visual_crops_ids = np.random.randint(0, TEST_SIZE, batch_size)
      visual_test_crops = test_data[visual_crops_ids, :]
      visual_target_crops = test_answ[visual_crops_ids, :]

      print("Training network")

      logs = open("models/logs.txt", "w+")
      logs.close()

      training_loss = 0.0

      for i in range(num_train_iters + 1):

          # Train PyNET model

          idx_train = np.random.randint(0, train_size, batch_size)

          phone_images = train_data[idx_train]
          dslr_images = train_answ[idx_train]

          # Random flips and rotations

          for k in range(batch_size):

              random_rotate = np.random.randint(1, 100) % 4
              phone_images[k] = np.rot90(phone_images[k], random_rotate)
              dslr_images[k] = np.rot90(dslr_images[k], random_rotate)
              random_flip = np.random.randint(1, 100) % 2

              if random_flip == 1:
                  phone_images[k] = np.flipud(phone_images[k])
                  dslr_images[k] = np.flipud(dslr_images[k])

          # Training step

          [loss_temp, temp] = sess.run([loss_generator, train_step_gen], feed_dict={input_: phone_images, target_: dslr_images})
          training_loss += loss_temp / eval_step

          if i % eval_step == 0:

              # Evaluate PyNET model

              test_losses = np.zeros((1, 6 if LEVEL < 4 else 5))

              for j in range(num_test_batches):

                  be = j * batch_size
                  en = (j+1) * batch_size

                  phone_images = test_data[be:en]
                  dslr_images = test_answ[be:en]

                  if LEVEL < 4:
                      losses = sess.run([loss_generator, loss_content, loss_mse, loss_psnr, loss_l1, loss_ms_ssim], \
                                      feed_dict={input_: phone_images, target_: dslr_images})
                  else:
                      losses = sess.run([loss_generator, loss_content, loss_mse, loss_psnr, loss_l1], \
                                        feed_dict={input_: phone_images, target_: dslr_images})

                  test_losses += np.asarray(losses) / num_test_batches

              if LEVEL < 4:
                  logs_gen = "step %d | training: %.4g, test: %.4g | content: %.4g, mse: %.4g, psnr: %.4g, l1: %.4g, " \
                            "ms-ssim: %.4g\n" % (i, training_loss, test_losses[0][0], test_losses[0][1],
                                                  test_losses[0][2], test_losses[0][3], test_losses[0][4], test_losses[0][5])
              else:
                  logs_gen = "step %d | training: %.4g, test: %.4g | content: %.4g, mse: %.4g, psnr: %.4g, l1: %.4g\n" % \
                        (i, training_loss, test_losses[0][0], test_losses[0][1], test_losses[0][2], test_losses[0][3], test_losses[0][4])
              print(logs_gen)

              # Save the results to log file

              logs = open("models/logs.txt", "a")
              logs.write(logs_gen)
              logs.write('\n')
              logs.close()

              # Save visual results for several test images

              bokeh_crops = sess.run(bokeh_img, feed_dict={input_: visual_test_crops, target_: dslr_images})

              idx = 0
              for crop in bokeh_crops:
                  if idx < 7:
                      before_after = np.hstack((
                                      np.float32(imageio.imresize(
                                          np.reshape(visual_test_crops[idx, :, :, 0:3] * 255, [PATCH_HEIGHT, PATCH_WIDTH, 3]),
                                                    [TARGET_HEIGHT, TARGET_WIDTH])) / 255.0,
                                      crop,
                                      np.reshape(visual_target_crops[idx], [TARGET_HEIGHT, TARGET_WIDTH, TARGET_DEPTH])))
                      imageio.imsave("results/pynet_img_" + str(idx) + "_level_" + str(LEVEL) + "_iter_" + str(i) + ".jpg",
                                  before_after)
                  idx += 1

              training_loss = 0.0

              # Saving the model that corresponds to the current iteration
              saver.save(sess, "models/pynet_level_" + str(LEVEL) + "_iteration_" + str(i) + ".ckpt", write_meta_graph=False)

          # Loading new training data
          if i % 1000 == 0:

              del train_data
              del train_answ
              train_data, train_answ = load_training_batch(dataset_dir, PATCH_WIDTH, PATCH_HEIGHT, DSLR_SCALE, train_size)


no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here
no errors here


KeyboardInterrupt: ignored